In [44]:
import numpy as np
import time
from sorted_row_collisions import *
from gpu_attempt_test import *
from hash import *

In [45]:
#test_matrix(hash_adjacency_matrix)

In [46]:
size = 4
matrix = np.random.randint(2, size=(size, size))
#print(matrix)
r_collisions = row_collisions(matrix)
r_diag = np.diag(r_collisions).reshape(1, -1).repeat(r_collisions.shape[0], axis=0)
#stacked_matrices = np.stack([r_collisions, r_diag], axis=2)
stacked_matrices = np.stack([r_diag, r_collisions], axis=2)
print(stacked_matrices)
stacked_matrices = np.sort(stacked_matrices, axis=1)

new_matrix = np.array([np.array([x,y,z]) for x, y, z in sorted(
    zip(stacked_matrices[:, :, 0].tolist(), stacked_matrices[:, :, 1].tolist(), matrix.tolist()))])

print(new_matrix)
new_matrix[:,2,:]

[[[2 2]
  [2 1]
  [3 1]
  [2 2]]

 [[2 1]
  [2 2]
  [3 2]
  [2 1]]

 [[2 1]
  [2 2]
  [3 3]
  [2 1]]

 [[2 2]
  [2 1]
  [3 1]
  [2 2]]]
[[[2 2 2 3]
  [1 1 2 2]
  [0 0 1 1]]

 [[2 2 2 3]
  [1 1 2 2]
  [0 1 0 1]]

 [[2 2 2 3]
  [1 1 2 2]
  [0 1 0 1]]

 [[2 2 2 3]
  [1 1 2 3]
  [1 0 1 1]]]


array([[0, 0, 1, 1],
       [0, 1, 0, 1],
       [0, 1, 0, 1],
       [1, 0, 1, 1]])

In [67]:
r_collisions = get_row_collisions(matrix)
r_collision_diag_indices = np.argsort(np.diag(r_collisions))
print(r_collision_diag_indices)
print('----------------')
print(r_collisions)
print('----------------')
sorted_diag_r_collisions = r_collisions[r_collision_diag_indices][:, r_collision_diag_indices]
print(sorted_diag_r_collisions)
print('----------------')
split_collisions = split_r_collisions(sorted_diag_r_collisions, unique_diags_list(sorted_diag_r_collisions))
print(split_collisions)
print('----------------')
collision_order_1 = np.lexsort(split_collisions.T)#new_order_of_rows(split_collisions)
print(collision_order)
print('----------------')
split_collisions = split_collisions[collision_order_1]
print(split_collisions)
print('----------------')
collision_order = new_order_of_rows(split_collisions)
print(collision_order[collision_order_1])
print('----------------')
diags = np.diag(r_collisions)
indecis = np.lexsort(np.stack([diags, collision_order]))
matrix[indecis]

[0 1 3 2]
----------------
[[2 1 1 2]
 [1 2 2 1]
 [1 2 3 1]
 [2 1 1 2]]
----------------
[[2 1 2 1]
 [1 2 1 2]
 [2 1 2 1]
 [1 2 1 3]]
----------------
[[1 2 2 1]
 [1 1 2 2]
 [1 2 2 1]
 [1 1 2 3]]
----------------
[0 0 2 3]
----------------
[[1 2 2 1]
 [1 2 2 1]
 [1 1 2 2]
 [1 1 2 3]]
----------------
[0 2 0 3]
----------------


array([[0, 1, 0, 1],
       [0, 0, 1, 1],
       [1, 0, 1, 1],
       [0, 1, 0, 1]])

In [51]:
r_collisions = get_row_collisions(matrix)
r_collisions

array([[2, 1, 1, 2],
       [1, 2, 2, 1],
       [1, 2, 3, 1],
       [2, 1, 1, 2]])

In [52]:
split_collisions = split_r_collisions(r_collisions, unique_diags_list(r_collisions))
split_collisions

array([[1, 2, 1, 2],
       [1, 2, 2, 1],
       [1, 2, 3, 1],
       [1, 2, 1, 2]])

In [49]:
collision_order = new_order_of_rows(split_collisions)
collision_order

array([1, 2, 0, 0], dtype=int64)

In [33]:
def new_order_of_rows(collision_matrix):
    prev_order = np.lexsort(collision_matrix.T)
    #print(prev_order)
    #print('----------------')
    #print(f"previous index: {prev_order}")
    prev_row = collision_matrix[prev_order][0]
    prev_index = 0
    for index, row in enumerate(collision_matrix[prev_order]):
        #print(row)
        if not np.allclose(prev_row, row):
            prev_index = index
            prev_row = row
        prev_order[index] = prev_order[prev_index]
    return prev_order

In [40]:
r_collisions = get_row_collisions(matrix)
collision_diags = np.diag(r_collisions)
decimal_val = get_decimal_edge_distance(matrix)

full_matrix = np.stack([collision_diags, decimal_val])
print(full_matrix)
print(np.lexsort(full_matrix))
#print(f"full matrix shape: {full_matrix.shape}")
#print(full_matrix)
#row_order = new_order_of_rows(full_matrix)
row_order = np.lexsort(full_matrix)

#print(f"row order: {row_order}")

#row_collisions = row_collisions[row_order][:, row_order]
matrix[row_order]

[[ 0  2  2  3]
 [ 0 15 22 22]]
[0 1 2 3]


array([[0, 0, 0, 0],
       [1, 0, 1, 0],
       [0, 1, 0, 1],
       [1, 1, 1, 0]])

In [49]:
matrix_row_collisions_split_then_sorted_1(matrix[row_order])

(array([[0, 0, 0, 0],
        [1, 0, 1, 0],
        [0, 1, 0, 1],
        [1, 1, 1, 0]]),
 array([[0],
        [4],
        [3],
        [6]]),
 array([[1],
        [1],
        [1],
        [1]]))

In [ ]:
#compress the large graph by finding its graph isomorphism, save the top-left part of the 2D matrix

In [42]:
def get_decimal_edge_distance(adjacency_matrix):
    return np.sum(np.matmul(
        adjacency_matrix,
        2 ** get_edge_distance_matrix(adjacency_matrix.shape[0], adjacency_matrix.shape[1])),
        axis=1)

In [43]:
def get_edge_distance_matrix(height, width):
    arr_1 = np.arange(width)
    arr_2 = np.arange(height).reshape(height, 1)

    repeated_arr_1 = np.tile(arr_1, (height, 1))
    repeated_arr_2 = np.tile(arr_2, (1, width))

    return np.minimum(repeated_arr_1, repeated_arr_2)

In [44]:
def get_score(adjacency_matrix):
    row_sum = np.sum(adjacency_matrix, axis=0)
    col_score = np.matmul(row_sum, adjacency_matrix.T)
    return col_score

In [45]:
#maybe sort by decimal last?
def matrix_row_collisions_split_then_sorted_1(matrix):
    matrix, row_collisions = get_matrix_and_row_collisions_sorted_by_row_collision_diagonals(matrix)

    diags = np.sum(row_collisions, axis=1)

    #get row collision order
    row_collisions = sort_chunked_row_collisions(row_collisions)
    #row_collision_order = custom_row_order(np.fliplr(row_collisions)).reshape(-1,1)
    row_collision_order = order_of_rows(row_collisions).reshape(-1,1) #since this gets sorted, this should be fine to order???

    return matrix, diags.reshape(-1,1), row_collision_order

In [46]:
def new_order_of_rows(collision_matrix):
    prev_order = np.lexsort(collision_matrix.T)
    #print(f"previous index: {prev_order}")
    prev_row = collision_matrix[prev_order][0]
    prev_index = 0
    for index, row in enumerate(collision_matrix[prev_order]):
        #print(row)
        if not np.allclose(prev_row, row):
            prev_index = index
            prev_row = row
        prev_order[index] = prev_order[prev_index]
    return prev_order

In [47]:
def new_hash(matrix):
    matrix, row_collisions, col_collisions = new_matrix_row_and_col_collisions(matrix)
    #matrix, row_collisions, col_collisions = new_matrix_row_and_col_collisions(matrix.T)
    #return matrix

    matrix = sort_adjacency_matrix_by_scores_and_decimal(matrix.T).T
    matrix = sort_adjacency_matrix_by_scores_and_decimal(matrix)
    return matrix

    #row sort starts
    row_collision_diags = np.diag(row_collisions).reshape(-1,1)
    decimal_matrix = get_row_bit_scores(matrix).reshape(-1,1)

    row_sorting_matrix = np.concatenate([row_collision_diags, row_collisions, decimal_matrix],axis=1)
    argsorted_rows_indecis = new_order_of_rows(row_sorting_matrix)
    matrix = matrix[argsorted_rows_indecis]

    #col sort starts
    col_collision_diags = np.diag(col_collisions).reshape(-1,1)
    decimal_matrix = get_row_bit_scores(matrix.T).reshape(-1,1)

    row_sorting_matrix = np.concatenate([col_collision_diags, col_collisions, decimal_matrix],axis=1)
    argsorted_rows_indecis = new_order_of_rows(row_sorting_matrix)
    matrix = matrix.T[argsorted_rows_indecis]

    return matrix.T

In [48]:
def new_matrix_row_and_col_collisions(matrix):
    matrix_T, col_collisions = new_get_matrix_and_row_collisions_sorted_by_row_collision_diagonals(matrix.T)
    matrix, row_collisions = new_get_matrix_and_row_collisions_sorted_by_row_collision_diagonals(matrix_T.T)

    #matrix = matrix_T.T
    return matrix, row_collisions, col_collisions


def new_get_matrix_and_row_collisions_sorted_by_row_collision_diagonals(matrix):
    row_collisions = get_row_collisions(matrix)
    collision_diags = np.diag(row_collisions)
    score = get_score(matrix)
    print(f"matrix: {matrix.shape}")
    decimal_val = get_decimal_edge_distance(matrix)

    full_matrix = np.stack([collision_diags, decimal_val])
    #print(f"full matrix shape: {full_matrix.shape}")
    #print(full_matrix)
    #row_order = new_order_of_rows(full_matrix)
    #row_order = np.lexsort(full_matrix)

    print(f"row order: {row_order}")

    row_collisions = row_collisions[row_order][:, row_order]
    new_matrix = matrix[row_order]
    return new_matrix, row_collisions

In [60]:
size = 4
matrix = np.random.randint(2, size=(size, size))
matrix

array([[0, 1, 0, 0],
       [0, 1, 1, 1],
       [1, 1, 0, 1],
       [1, 0, 1, 0]])

In [61]:
new_matrix, _, _ = new_matrix_row_and_col_collisions(matrix.T)
new_matrix, _, _ = new_matrix_row_and_col_collisions(new_matrix.T)
new_matrix

matrix; (4, 4)
row order: [0 3 2 1]
matrix; (4, 4)
row order: [0 2 3 1]
matrix; (4, 4)
row order: [0 1 2 3]
matrix; (4, 4)
row order: [1 0 2 3]


array([[1, 1, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 1, 1],
       [0, 1, 1, 1]])

In [55]:
hash_adjacency_matrix_helper(hash_adjacency_matrix_helper(matrix.T).T)

array([[0, 0, 1, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 1],
       [0, 1, 1, 1]])

In [147]:
test_matrix(new_hash)

num of matrices: 65536
0
[[0 0 0 0]
 [1 1 0 0]
 [0 0 0 1]
 [0 0 1 1]]
[[0 0 0 0]
 [0 0 0 1]
 [0 1 1 0]
 [1 0 0 1]]


AssertionError: 

In [9]:
test_matrix(hash_adjacency_matrix)

num of matrices: 65536
0
[[0 0 0 0]
 [1 1 0 0]
 [0 0 0 1]
 [0 0 1 1]]
[[0 0 0 0]
 [0 0 0 1]
 [0 1 1 0]
 [1 0 0 1]]


AssertionError: 

In [8]:
def get_order_of_chunked_row_collisions(collision_matrix):
    chunked_row_collisions = sort_chunked_row_collisions(collision_matrix)
    chunked_row_order = new_order_of_rows(chunked_row_collisions)#.reshape(-1,1) #this is still wrong...
    #print(f"chunked collisions:")
    #print(chunked_row_collisions)
    return chunked_row_order

In [9]:
def mini_hash(matrix):
    matrix, row_collisions, col_collisions = matrix_row_and_col_collisions(matrix)

    #print("-------------matrix-----------")
    #print(matrix)
    #print("---------row collisions-----------")
    #print(row_collisions)
    #print("---------col collisions-----------")
    #print(col_collisions)


    #start row sorts
    chunked_row_order = get_order_of_chunked_row_collisions(row_collisions).reshape(-1,1)
    decimal_matrix = get_row_bit_scores(matrix).reshape(-1,1)

    row_sorting_matrix = np.concatenate([chunked_row_order, decimal_matrix],axis=1)
    #print(f"row sorting matrix:")
    #print(row_sorting_matrix)
    argsorted_rows_indecis = new_order_of_rows(np.fliplr(row_sorting_matrix))#order_of_rows(row_sorting_matrix)

    #print(f"arg sorted rows: {argsorted_rows_indecis.reshape(-1)}")

    matrix = matrix[argsorted_rows_indecis]
    #print("matrix after row sorts")
    #print(matrix)

    #col sort starts
    chunked_col_order = get_order_of_chunked_row_collisions(col_collisions).reshape(-1,1)
    decimal_matrix = get_row_bit_scores(matrix.T).reshape(-1,1)

    col_sorting_matrix = np.concatenate([chunked_col_order, decimal_matrix],axis=1)
    #print(f"col sorting matrix:")
    #print(col_sorting_matrix)
    argsorted_cols_indecis = new_order_of_rows(np.fliplr(col_sorting_matrix))#order_of_rows(row_sorting_matrix)
    #print(f"arg sorted cols: {argsorted_cols_indecis.reshape(-1)}")
    matrix = matrix[:, argsorted_cols_indecis]

    #print("matrix after col sorts")
    #print(matrix)
    #print("--------------------")
    #print(row_collisions)
    #print("--------------------")
    #print(col_collisions)

    return matrix

In [32]:
def mini_hash_2_1(matrix):
    matrix, row_collisions, col_collisions = matrix_row_and_col_collisions(matrix)

    #start row sorts
    chunked_row_order = get_order_of_chunked_row_collisions(row_collisions).reshape(-1,1)
    decimal_matrix = get_row_bit_scores(matrix).reshape(-1,1)

    row_sorting_matrix = np.concatenate([decimal_matrix, chunked_row_order],axis=1)
    argsorted_rows_indecis = new_order_of_rows(row_sorting_matrix)

    matrix = matrix[argsorted_rows_indecis]

    #col sort starts
    chunked_col_order = get_order_of_chunked_row_collisions(col_collisions).reshape(-1,1)
    decimal_matrix = get_row_bit_scores(matrix.T).reshape(-1,1)

    col_sorting_matrix = np.concatenate([decimal_matrix, chunked_col_order],axis=1)
    argsorted_cols_indecis = new_order_of_rows(col_sorting_matrix)
    matrix = matrix[:, argsorted_cols_indecis]

    return matrix

In [11]:
def mini_hash_2(matrix):
    return mini_hash(mini_hash(matrix))

def mini_hash_2_2(matrix):
    return mini_hash_2_1(mini_hash_2_1(matrix))

In [12]:
def mini_hash_3(matrix):
    matrix, row_collisions, col_collisions = matrix_row_and_col_collisions(matrix)

    #start row sorts
    chunked_row_order = np.argsort(get_order_of_chunked_row_collisions(row_collisions))
    chunked_col_order = np.argsort(get_order_of_chunked_row_collisions(col_collisions))
    matrix = matrix[chunked_row_order][:, chunked_col_order]

    matrix = sort_adjacency_matrix_by_scores_and_decimal(matrix.T).T
    matrix = sort_adjacency_matrix_by_scores_and_decimal(matrix)

    return matrix

In [102]:
a = format_array('[[0 0 0 1] [1 0 0 1] [0 1 1 0] [0 0 0 0]]')
b = format_array('[[0 0 0 0] [0 0 0 1] [1 1 0 0] [0 0 1 1]]')
c = format_array('[[0 0 0 1] [0 1 1 0] [1 0 0 1] [0 0 0 0]]')

In [35]:
a = format_array('[[0 0 0 0] [0 0 1 0] [1 0 0 0] [0 1 0 1]]')
b = format_array('[[0 0 0 0] [1 0 0 0] [0 0 1 0] [0 1 0 1]]')
c = b.T

In [36]:
a

array([[0, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 1]])

In [49]:
matrix = np.random.randint(2, size=(4, 4))#a
print("matrix")
print(matrix)
r_collisions = row_collisions(matrix)
r_diag = np.diag(r_collisions).reshape(1, -1).repeat(r_collisions.shape[0], axis=0)
stacked_matrices = np.stack([r_collisions, r_diag], axis=2)

print(f"row collisions")
print(r_collisions)

print(f"stacked matrices")
print(f"shape: {stacked_matrices.shape}")
print(stacked_matrices)

stacked_matrices = np.sort(stacked_matrices, axis=1)

print(f"shape: {stacked_matrices.shape}")
print(f"sorted stacked matrices")
print(stacked_matrices)

print(f"at 0")
print(stacked_matrices[:, :, 0])
print(f"at 1")
print(stacked_matrices[:, :, 1])

matrix = np.array([x for _, _, x in sorted(
    zip(stacked_matrices[:, :, 0].tolist(), stacked_matrices[:, :, 1].tolist(), matrix.tolist()))])

#print("order of rows")
#print(new_order_of_rows(stacked_matrices))


matrix

matrix
[[0 1 1 1]
 [0 0 1 1]
 [0 0 0 1]
 [0 0 1 0]]
row collisions
[[3 2 1 1]
 [2 2 1 1]
 [1 1 1 0]
 [1 1 0 1]]
stacked matrices
shape: (4, 4, 2)
[[[3 3]
  [2 2]
  [1 1]
  [1 1]]

 [[2 3]
  [2 2]
  [1 1]
  [1 1]]

 [[1 3]
  [1 2]
  [1 1]
  [0 1]]

 [[1 3]
  [1 2]
  [0 1]
  [1 1]]]
shape: (4, 4, 2)
sorted stacked matrices
[[[1 1]
  [1 1]
  [2 2]
  [3 3]]

 [[1 1]
  [1 1]
  [2 2]
  [2 3]]

 [[0 1]
  [1 1]
  [1 2]
  [1 3]]

 [[0 1]
  [1 1]
  [1 2]
  [1 3]]]
at 0
[[1 1 2 3]
 [1 1 2 2]
 [0 1 1 1]
 [0 1 1 1]]
at 1
[[1 1 2 3]
 [1 1 2 3]
 [1 1 2 3]
 [1 1 2 3]]
order of rows
[[2 3 0 1]
 [0 1 2 3]
 [2 3 0 1]
 [2 3 1 0]]


array([[0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 1, 1],
       [0, 1, 1, 1]])

In [15]:
print(a)
matrix = mini_hash(a)
matrix

[[0 0 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 1 0 1]]


array([[0, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 1]])

In [16]:
print(c)
matrix = mini_hash_3(a)
matrix

[[0 1 0 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 0 1]]


array([[0, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 1]])

In [17]:
matrix = mini_hash_3(mini_hash_3(a))
matrix

array([[0, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 1]])

In [10]:
test_matrix(hash_adjacency_matrix)

num of matrices: 65536
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000


KeyboardInterrupt: 

In [33]:
test_matrix(mini_hash_2_1)

num of matrices: 65536
0
[[0 0 0 0]
 [0 0 0 0]
 [1 0 1 0]
 [0 1 1 1]]
[[0 0 0 0]
 [0 0 0 0]
 [0 0 1 1]
 [1 1 1 0]]


AssertionError: 

In [19]:
test_matrix(mini_hash_2_2)

num of matrices: 65536
0
[[0 0 0 0]
 [0 0 0 0]
 [1 0 1 0]
 [0 1 1 1]]
[[0 0 0 0]
 [0 0 0 0]
 [0 0 1 1]
 [1 1 1 0]]


AssertionError: 

In [29]:
test_matrix(full_hash_1)

num of matrices: 65536
0


IndexError: index 4 is out of bounds for axis 0 with size 1

In [28]:
test_matrix(mini_hash_2)

num of matrices: 65536
0
[[0 0 0 0]
 [0 0 0 0]
 [1 0 1 0]
 [0 1 1 1]]
[[0 0 0 0]
 [0 0 0 0]
 [0 0 1 1]
 [1 1 1 0]]


AssertionError: 

In [29]:
test_matrix(mini_hash_3)

num of matrices: 65536
0
1000


KeyboardInterrupt: 

In [28]:
test_matrix(hash_adjacency_matrix)

num of matrices: 65536
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000


KeyboardInterrupt: 

In [20]:
large_size = 300
large_matrix = np.random.randint(2, size=(large_size, large_size))

In [53]:
size = 5
matrix = np.random.randint(2, size=(size, size))
matrix, row_collisions, col_collisions = matrix_row_and_col_collisions(matrix)
collision_matrix = row_collisions
collision_matrix

array([[2, 1, 1, 2, 2],
       [1, 3, 1, 2, 3],
       [1, 1, 3, 3, 2],
       [2, 2, 3, 4, 3],
       [2, 3, 2, 3, 4]])

In [54]:
chunked_row_collisions = sort_chunked_row_collisions(collision_matrix)
#chunked_row_order = new_order_of_rows(chunked_row_collisions)
chunked_row_collisions

array([[2, 1, 1, 2, 2],
       [1, 1, 3, 2, 3],
       [1, 1, 3, 2, 3],
       [2, 2, 3, 3, 4],
       [2, 2, 3, 3, 4]])

In [55]:
new_order = new_order_of_rows(chunked_row_collisions)
new_order

array([0, 1, 1, 3, 3], dtype=int64)

In [39]:
matrix_1, row_collisions_1, col_collisions_1 = matrix_row_and_col_collisions(matrix[new_order])
row_collisions_1

array([[2, 0, 2, 2, 2],
       [0, 3, 1, 1, 2],
       [2, 1, 3, 2, 2],
       [2, 1, 2, 3, 3],
       [2, 2, 2, 3, 4]])

In [42]:
sort_chunked_row_collisions(row_collisions_1)

array([[2, 0, 2, 2, 2],
       [0, 1, 1, 3, 2],
       [2, 1, 2, 3, 2],
       [2, 1, 2, 3, 3],
       [2, 2, 2, 3, 4]])

In [43]:
get_score(row_collisions)

array([ 88, 107, 120,  82, 138])

In [44]:
get_score(row_collisions_1)

array([ 88,  82, 107, 120, 138])

In [50]:
np.sum(row_collisions_1, axis=1)

array([ 8,  7, 10, 11, 13])

In [57]:
matrix

array([[0, 0, 1, 0, 1],
       [1, 0, 1, 1, 0],
       [0, 1, 0, 1, 1],
       [0, 1, 1, 1, 1],
       [1, 0, 1, 1, 1]])

In [56]:
r_collisions = row_collisions
r_diag = np.diag(r_collisions).reshape(1, -1).repeat(r_collisions.shape[0], axis=0)
stacked_matrices = np.stack([r_collisions, r_diag], axis=2)

stacked_matrices = np.sort(stacked_matrices, axis=1)

np.array([x for _, _, x in sorted(zip(stacked_matrices[:, :, 0].tolist(), stacked_matrices[:, :, 1].tolist(), matrix.tolist()))])

array([[0, 0, 1, 0, 1],
       [0, 1, 0, 1, 1],
       [1, 0, 1, 1, 0],
       [0, 1, 1, 1, 1],
       [1, 0, 1, 1, 1]])

In [27]:
%timeit hash_adjacency_matrix(large_matrix)

470 ms ± 7.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%timeit test_matrix(mini_hash_3)

num of matrices: 65536
0
1000
2000
3000
4000
[[0 0 1 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 1 0 1]]
[[0 1 0 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 1 1]]


AssertionError: 

In [ ]:
%timeit test_matrix(mini_hash_2)

In [32]:
collision_matrix = np.array([[0, 0, 1, 1],
                   [0, 1, 1, 0],
                   [0, 1, 1, 0],
                   [0, 0, 1, 2]])
expected_array = [1, 1, 0, 2]

In [43]:
collision_matrix

array([[0, 0, 1, 1],
       [0, 1, 1, 0],
       [0, 1, 1, 0],
       [0, 0, 1, 2]])

In [44]:
collision_matrix.T

array([[0, 0, 0, 0],
       [0, 1, 1, 0],
       [1, 1, 1, 1],
       [1, 0, 0, 2]])

previous index: [1 2 0 3]
[0 1 1 0]
[0 1 1 0]
[0 0 1 1]
[0 0 1 2]


array([1, 1, 0, 3], dtype=int64)